In [ ]:
from nipype import Node, Workflow, IdentityInterface, Function, MapNode,SelectFiles, DataSink
from nipype.interfaces import fsl
from nipype.algorithms.modelgen import SpecifyModel
import numpy as np
import os

In [ ]:
def sort_inputs(copes,varcopes,num_of_runs):
    
    copes.sort(key = lambda x: x.split("/")[-1])
    cope_list =  [copes[i:i + num_of_runs] for i in range(0, len(copes), num_of_runs)] 

    varcopes.sort(key = lambda x: x.split("/")[-1])
    varcope_list =  [varcopes[i:i + num_of_runs] for i in range(0, len(varcopes), num_of_runs)] 
    
    return cope_list,varcope_list

In [ ]:
data = "/media/paradeisios/UUI/fmri/flanker/output/first_level"
output = "/media/paradeisios/UUI/fmri/flanker/output/second_level"
mask_file = "/home/paradeisios/fsl/data/standard/MNI152_T1_2mm_brain_mask.nii.gz"

num_of_subjects = 26
num_of_runs = 2

subject_list = ["{:02d}".format(i) for i in range(1,num_of_subjects+1)]

In [ ]:
infosource = Node(IdentityInterface(fields=["subject_list"]), name="infosource")
infosource.iterables = [('subject_list', subject_list)]

In [ ]:
templates = {"copes":     "modelestimate/{subject_list}/modelestimate_run_*/cope*.nii.gz",
             "varcopes":  "modelestimate/{subject_list}/modelestimate_run_*/varcope*.nii.gz",
             "dof_files" : "modelestimate/{subject_list}/modelestimate_run_*/dof"}

selectfiles = Node(SelectFiles(templates),name="selectfiles")
selectfiles.inputs.base_directory = data

In [ ]:
substitutions = []
substitutions += [("_subject_list_{:02d}".format(i), "{:02d}".format(i)) for i in range(1,num_of_subjects+1)]
substitutions += [("_fixed_eff_model{:d}".format(i), "fixed_eff_model_{:d}".format(i+1)) for i in range(0,num_of_subjects+1)]
substitutions += [("_copemerge", "copemerge")]
substitutions += [("_varcopemerge", "varcopemerge")]
datasink = Node(DataSink(base_directory=output),name="datasink")
datasink.inputs.substitutions = substitutions
datasink.output = output

In [ ]:
sort_copes_varcopes = Node(Function(input_names=["copes","varcopes","num_of_runs"],
                                  output_names=["cope_list","varcope_list"],
                                  function=sort_inputs),
                                  name='sort_copes_varcopes')

sort_copes_varcopes.inputs.num_of_runs = num_of_runs

In [ ]:
copemerge = MapNode(fsl.Merge(),iterfield=['in_files'],name="copemerge")
copemerge.inputs.dimension = "t"

varcopemerge = MapNode(fsl.Merge(),iterfield=['in_files'],name="varcopemerge")
varcopemerge.inputs.dimension = "t"

level2model = Node(interface=fsl.L2Model(), name='l2model')
level2model.inputs.num_copes = num_of_runs

fixed_eff_model = MapNode(interface=fsl.FLAMEO(),name="fixed_eff_model",iterfield=['cope_file', 'var_cope_file'])
fixed_eff_model.inputs.run_mode='fe'
fixed_eff_model.inputs.mask_file= mask_file

In [ ]:
fixed_effects_analysis = Workflow(name="fixed_effects_analysis")

fixed_effects_analysis.connect(infosource,"subject_list",datasink,"container")
fixed_effects_analysis.connect(infosource,"subject_list",selectfiles,"subject_list")
fixed_effects_analysis.connect(selectfiles,"copes",sort_copes_varcopes,"copes")
fixed_effects_analysis.connect(selectfiles,"varcopes",sort_copes_varcopes,"varcopes")

fixed_effects_analysis.connect(sort_copes_varcopes,"cope_list",copemerge,"in_files")
fixed_effects_analysis.connect(sort_copes_varcopes,"varcope_list",varcopemerge,"in_files")

fixed_effects_analysis.connect(copemerge,"merged_file",fixed_eff_model,"cope_file")
fixed_effects_analysis.connect(varcopemerge,"merged_file",fixed_eff_model,"var_cope_file")

    
fixed_effects_analysis.connect(level2model,"design_mat",fixed_eff_model,"design_file")   
fixed_effects_analysis.connect(level2model,"design_con",fixed_eff_model,"t_con_file") 
fixed_effects_analysis.connect(level2model,"design_grp",fixed_eff_model,"cov_split_file") 

In [ ]:
fixed_effects_analysis.write_graph(graph2use='flat', format='png', simple_form=True,
                          dotfilename= os.path.join(output,"fixed_effects_analysis_workflow.dot"))

In [ ]:
fixed_effects_analysis.connect(selectfiles,"copes",datasink,"copes")
fixed_effects_analysis.connect(selectfiles,"varcopes",datasink,"varcopes")
fixed_effects_analysis.connect(selectfiles,"dof_files",datasink,"dof_files")

fixed_effects_analysis.connect(copemerge,"merged_file",datasink,"merge.@merged_cope_file")
fixed_effects_analysis.connect(varcopemerge,"merged_file",datasink,"merge.@merged_varcope_file")

fixed_effects_analysis.connect(level2model,"design_mat",datasink,"model.@design_mat")

fixed_effects_analysis.connect(fixed_eff_model,"copes",datasink,"fixed_effects.@copes")
fixed_effects_analysis.connect(fixed_eff_model,"var_copes",datasink,"fixed_effects.@var_copes")
fixed_effects_analysis.connect(fixed_eff_model,"weights",datasink,"fixed_effects.@weights")
fixed_effects_analysis.connect(fixed_eff_model,"zstats",datasink,"fixed_effects.@zstats")
fixed_effects_analysis.connect(fixed_eff_model,"res4d",datasink,"fixed_effects.@res4d")
fixed_effects_analysis.connect(fixed_eff_model,"tstats",datasink,"fixed_effects.@tstats")

In [ ]:
fixed_effects_analysis.run(plugin='MultiProc', plugin_args={'n_procs' : 1})